In [1]:
import pyarrow.parquet as pa
import pandas as pd
import pyarrow.csv as csv
import numpy as np

In [2]:
## read parquet
dataset = pa.ParquetDataset('recomm/')
table = dataset.read()

In [3]:
table

pyarrow.Table
customer_id_index: int32 not null
article_id_index: list<element: int32>
  child 0, element: int32
----
customer_id_index: [[12,65,76,81,122,...,10345,10362,10433,10502,10506],[12,65,76,81,122,...,10345,10362,10416,10502,10506],...,[32,106,116,198,203,...,10337,10353,10386,10491,10521],[67,138,172,180,315,...,10189,10275,10324,10383,10399]]
article_id_index: [[[1867,7955,5605,6036,5662,...,6855,5908,7367,4912,7479],[3503,5605,1867,2891,5620,...,2550,7930,4209,2467,2726],...,[2891,1867,5782,6289,5605,...,7789,4326,6599,4817,3741],[5620,5605,2891,1867,5662,...,4817,6599,7789,6855,7008]],[[1938,7955,1867,5662,6722,...,6965,4912,7479,7814,2726],[1938,1867,5782,6289,5662,...,6036,6599,4038,6965,1509],...,[1938,7955,5662,6722,1867,...,6965,4912,7479,1434,334],[1938,1867,7955,5662,6289,...,6964,7814,6888,7040,737]],...,[[1867,5605,5620,1938,2891,...,5782,6289,5778,6965,6599],[1867,5605,1938,5662,5620,...,4326,6964,5908,2868,5778],...,[1867,5605,5662,1938,2891,...,5371,2868,5058,

In [4]:
## transform parquet to pandas
recommendation_customer = table.to_pandas()

In [5]:
recommendation_customer

,customer_id_index,article_id_index
0,12,"[1867, 7955, 5605, 6036, 5662, 4699, 6722, 246..."
1,65,"[3503, 5605, 1867, 2891, 5620, 1938, 6017, 696..."
2,76,"[1867, 5620, 2891, 5782, 6289, 5605, 3503, 246..."
3,81,"[1867, 5605, 2891, 4817, 6599, 3503, 5662, 469..."
4,122,"[1867, 5605, 5662, 1938, 3503, 2891, 5620, 469..."
...,...,...
10037,10189,"[1867, 1938, 5662, 5605, 5782, 6289, 5620, 795..."
10038,10275,"[1867, 5662, 2467, 5605, 5620, 5908, 7955, 469..."
10039,10324,"[1938, 7955, 5605, 2891, 3503, 5620, 5662, 186..."
10040,10383,"[1938, 1867, 6036, 5605, 5620, 3503, 7008, 289..."


In [6]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

groupproject = SparkSession.builder.appName("HM_Recommendations").config("spark.sql.files.maxPartitionBytes", 5000000).getOrCreate()
spark = SparkSession(groupproject)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 10:41:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
transaction = spark.read.option("header",True) \
              .csv("transactions_train.csv")
transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [8]:
## To prevent data problems, repeat the Dataset check and data choose steps in PySpark
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
print(min_date, max_date)

datahm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
datahm = datahm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
datahm = datahm.withColumn('year', year(col('date')))
datahm = datahm.withColumn('month', month(col('date')))
datahm = datahm.withColumn('day', date_format(col('date'), "d"))

datahm = datahm[datahm['year'] == 2020]
datahm = datahm[datahm['month'] == 9]
datahm = datahm[datahm['day'] == 22]
transaction.unpersist()

# Prepare the dataset
datahm = datahm.groupby('customer_id', 'article_id').count()
datahm.show()

2018-09-20 2020-09-22


+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|00f7bc5c0df4c615b...|0780418013|    1|
|02094817e46f3b692...|0791587001|    1|
|0333e5dda0257e9f4...|0839332002|    2|
|07c7a1172caf8fb97...|0573085043|    1|
|081373184e601470c...|0714790020|    1|
|09bec2a61046ccbea...|0860336002|    1|
|0be4f1ecce204ee32...|0573085028|    1|
|0c4b30343292b5101...|0918522001|    1|
|0e10e02358875468b...|0579541001|    1|
|0fc371e67e61a31d7...|0907170001|    1|
|10817b19177f6a53e...|0718278019|    1|
|10ac90988da6052dd...|0934212003|    1|
|14a298482fa7f9d52...|0894353002|    1|
|14f4b1b17991c32d2...|0803685001|    1|
|1601fa3c3f39aa623...|0730683001|    1|
|164e1a251f0e3d764...|0831267001|    1|
|165d2c0b0128d5619...|0909081004|    1|
|166546028742fe655...|0767423013|    1|
|189b7275c513a84c8...|0877711001|    1|
|1918933afff08e955...|0914672001|    1|
+--------------------+----------+-----+
only showing top 20 rows



In [9]:
userId_count = datahm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|30b6056bacc5f5c9d...|   28|
|5e8fb4d457fdffc61...|   28|
|dc1b173e541f8d3c1...|   27|
|6335d496ef463bc40...|   25|
|1796e87fd2e88932b...|   25|
|f50287d9cf052d4b4...|   24|
|54e8ebd39543b5a4d...|   23|
|fd5ce8716faf00f6a...|   23|
|850ec77661a417d27...|   22|
|ad3663a848dccbdda...|   21|
|32f3a6a7ce63d302c...|   21|
|b606fe5786c00151a...|   21|
|298523b6637340717...|   21|
|b49647f84a99ced53...|   21|
|fc783381f1ea2174c...|   21|
|a08e284bb18add2d7...|   21|
|383e1b07e2c1fe169...|   21|
|3ca77aab50ae4532b...|   20|
|2a721767cd9864ed5...|   20|
|af5166e0f89b0d433...|   19|
+--------------------+-----+
only showing top 20 rows



In [10]:
articleId_count = datahm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

+----------+-----+
|article_id|count|
+----------+-----+
|0924243002|   91|
|0918522001|   88|
|0866731001|   78|
|0751471001|   75|
|0448509014|   73|
|0714790020|   72|
|0762846027|   68|
|0928206001|   67|
|0893432002|   66|
|0918292001|   65|
|0915529005|   64|
|0788575004|   63|
|0915529003|   63|
|0863583001|   60|
|0930380001|   59|
|0573085028|   59|
|0919273002|   58|
|0850917001|   57|
|0573085042|   56|
|0874110016|   53|
+----------+-----+
only showing top 20 rows



In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(datahm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(datahm).transform(datahm)
transformed.show()

22/11/01 10:44:33 WARN DAGScheduler: Broadcasting large task binary with size 1210.8 KiB
+--------------------+----------+-----+----------------+-----------------+
|         customer_id|article_id|count|article_id_index|customer_id_index|
+--------------------+----------+-----+----------------+-----------------+
|00f7bc5c0df4c615b...|0780418013|    1|          2237.0|            783.0|
|02094817e46f3b692...|0791587001|    1|            35.0|            785.0|
|0333e5dda0257e9f4...|0839332002|    2|           732.0|           4098.0|
|07c7a1172caf8fb97...|0573085043|    1|            44.0|           1702.0|
|081373184e601470c...|0714790020|    1|             5.0|           4146.0|
|09bec2a61046ccbea...|0860336002|    1|          2368.0|           6792.0|
|0be4f1ecce204ee32...|0573085028|    1|            14.0|            799.0|
|0c4b30343292b5101...|0918522001|    1|             1.0|           6825.0|
|0e10e02358875468b...|0579541001|    1|            53.0|           2689.0|
|0fc371e67e

In [12]:
## select the column from the original dataset
df_trans = transformed.select(transformed['article_id'],transformed['article_id_index']
                        ,transformed['customer_id'],transformed['customer_id_index'])
df_trans = df_trans.toPandas()
df_trans

22/11/01 10:45:03 WARN DAGScheduler: Broadcasting large task binary with size 1207.7 KiB


,article_id,article_id_index,customer_id,customer_id_index
0,0780418013,2237.0,00f7bc5c0df4c615b2502a2c2e9ef9eff988c81dec2e5e...,783.0
1,0791587001,35.0,02094817e46f3b692149b06cf9577e42848c2294e78598...,785.0
2,0839332002,732.0,0333e5dda0257e9f498be52f1e569bfae576caed0cbdcd...,4098.0
3,0573085043,44.0,07c7a1172caf8fb9784b28e51b25b985ab6a1ec7ce923e...,1702.0
4,0714790020,5.0,081373184e601470cc9911f33d3eeebc6f33ed79222573...,4146.0
...,...,...,...,...
29481,0817150004,851.0,f8156f726aeaf44e90c1988837e13c9c9974ee19009e5b...,2594.0
29482,0893432005,96.0,f825202d015506981dc42c53afb7b56a36e05c85b67886...,1657.0
29483,0799754001,5499.0,faed38aaccd80db66f5a1581fe99af84e79fe398c91899...,10465.0
29484,0928206001,7.0,fcffcb9777aab7a53e3b382a840958d800e6d53bdd8a20...,4052.0


In [13]:
dict1 =dict(zip(df_trans['article_id_index'],df_trans['article_id']))
dict2 = dict(zip(df_trans['customer_id_index'],df_trans['customer_id']))
## select 5 articles for each customer
recommendation_customer['article_id'] = recommendation_customer['article_id_index'].map(
    lambda x: [dict1[y] for y in x if y in dict1][:5])
recommendation_customer['customer_id']=recommendation_customer['customer_id_index'].map(dict2)
recommendation_customer.drop(['customer_id_index','article_id_index'], axis = 1)
recommendation = recommendation_customer[['customer_id','article_id']]
recommendation

,customer_id,article_id
0,a08e284bb18add2d71129f0c9bf6a1b4e7a16ae4af54a4...,"[0857030003, 0942955002, 0807362001, 083437800..."
1,012611429367ef74cb9008dcf0069284501071e90717e5...,"[0860820001, 0807362001, 0857030003, 067405201..."
2,b082866c41c9301903250bb9b1a93083bbd741ec4bfcb5...,"[0857030003, 0808628001, 0674052013, 081741600..."
3,071462f2e395227659c39d35f6692bbf2854cd78cf53c9...,"[0857030003, 0807362001, 0674052013, 071139900..."
4,4ebaab0fab59c10a4aebc458de70477499a356716e606e...,"[0857030003, 0807362001, 0810818001, 089421000..."
...,...,...
10037,e8e625a95270f99feadfd45242b504d0f2e57facb956d3...,"[0857030003, 0894210001, 0810818001, 080736200..."
10038,eeed0202a669f67ba050545db289842e9a9f24d1c62308...,"[0857030003, 0810818001, 0883684002, 080736200..."
10039,f1e5da9d758bc02250442977641796133c6018c2474eb2...,"[0894210001, 0942955002, 0807362001, 067405201..."
10040,f5bd9cb67bcd2bdf3358fdf52c5afba8d8574571faf0f8...,"[0894210001, 0857030003, 0834378002, 080736200..."


In [15]:
recommendation.to_csv('recommendation.csv',index=False)